In [7]:
import pyterrier as pt
from experiments_helper import time_fct

if not pt.started():
    pt.init()

In [8]:
from pyterrier.measures import RR, nDCG, MAP

eval_metrics = [RR @ 10, nDCG @ 10, MAP @ 100]

Download ms marco index

In [6]:
dataset_name = "msmarco_document"
index_variant = 'terrier_stemmed'

dataset = pt.datasets.get_dataset(dataset_name)

indexref = dataset.get_index(index_variant)

retriever = bm25 = pt.BatchRetrieve(indexref, wmodel="BM25")

data.inverted.bf:   0%|          | 4.54M/1.35G [00:02<11:36, 2.07MiB/s]


KeyboardInterrupt: 

Run downloaded index

In [ ]:
dataset_name = "msmarco_document"
index_variant = 'terrier_stemmed'

retriever = pt.BatchRetrieve.from_dataset(dataset_name, index_variant, wmodel='BM25')

data.direct.bf: 100%|██████████| 1.29G/1.29G [01:46<00:00, 13.0MiB/s]
data.document.fsarrayfile: 100%|██████████| 52.1M/52.1M [00:03<00:00, 16.9MiB/s]
data.inverted.bf:  12%|█▏        | 171M/1.35G [00:10<01:47, 11.8MiB/s] 

In [8]:
from experiments_helper import run_single_experiment_name

dataset_test_name = "irds:msmarco-document-v2/trec-dl-2019"
time_fct(run_single_experiment_name, retriever, dataset_test_name, eval_metrics, dataset_name + ": BM25")

[INFO] [starting] https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz
[INFO] [finished] https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz: [00:00] [4.28kB] [4.70MB/s]
[INFO] [starting] https://msmarco.z22.web.core.windows.net/msmarcoranking/docv2_trec2019_qrels.txt.gz               
[INFO] [finished] https://msmarco.z22.web.core.windows.net/msmarcoranking/docv2_trec2019_qrels.txt.gz: [00:00] [107kB] [231kB/s]


Experiment took 70.998 seconds to execute.


,name,RR@10,nDCG@10,AP@100
0,msmarcov2_document: BM25,0.536074,0.248707,0.090859


### Download the passage index( other option than the dataset.get_index(index_variant)) one

In [3]:
dataset_name = "msmarco_passage"
index_variant = 'terrier_stemmed'

retriever = pt.BatchRetrieve.from_dataset(dataset_name, index_variant, wmodel='BM25')

In [6]:
from experiments_helper import run_single_experiment_name

dataset_test_name = "irds:msmarco-passage/trec-dl-2019"
time_fct(run_single_experiment_name, retriever, dataset_test_name, eval_metrics, dataset_name + ": BM25")

[INFO] Please confirm you agree to the MSMARCO data usage agreement found at <http://www.msmarco.org/dataset.aspx>
[INFO] [starting] https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz
[INFO] [finished] https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz: [00:00] [4.28kB] [4.22MB/s]
[INFO] [starting] https://trec.nist.gov/data/deep/2019qrels-pass.txt                                                
[INFO] [finished] https://trec.nist.gov/data/deep/2019qrels-pass.txt: [00:00] [187kB] [422kB/s]


Experiment took 12.249 seconds to execute.


,name,RR@10,nDCG@10,AP@100
0,msmarco_passage: BM25,0.79438,0.47954,0.290692


### Retrieve index( the one for passage) from memory given absolute path(this is where dataset.get_index(index_variant) stores it and I would not find an option for changing the path) and run an experiment. ! Fill in your absolute path

In [27]:
from experiments_helper import run_single_experiment_name

index_absolute_path = "/home/anistor/.pyterrier/corpora/msmarco_passage/index/terrier_stemmed"


index = pt.IndexFactory.of(index_absolute_path)

bm25_stored = pt.BatchRetrieve(index, wmodel="BM25")

dataset_name = "msmarco_passage"

dataset_test_name = "irds:msmarco-passage/trec-dl-2019"
time_fct(run_single_experiment_name, bm25_stored, dataset_test_name, eval_metrics, dataset_name + ": BM25")


Experiment took 10.578 seconds to execute.


,name,RR@10,nDCG@10,AP@100
0,msmarco_passage: BM25,0.79438,0.47954,0.290692


In [7]:
from pathlib import Path
from gte_base_en_encoder import GTEBaseDocumentEncoder
import torch
from fast_forward import OnDiskIndex, Mode, Indexer

q_encoder = GTEBaseDocumentEncoder("Alibaba-NLP/gte-base-en-v1.5")
d_encoder = GTEBaseDocumentEncoder(
    "Alibaba-NLP/gte-base-en-v1.5",
    device="cuda:0" if torch.cuda.is_available() else "cpu",
)

In [11]:
def docs_iter(dataset, limit=1000):
    # Iterate over the documents and yield up to 'limit' documents
    for count, d in enumerate(dataset.get_corpus_iter()):
        if count >= limit:
            break
        yield {"doc_id": d["docno"], "text": d["text"]}

In [9]:
def format_dataset_name(text):
    if ':' in text:
        return text.split(':')[0] + "_" + text.split(':')[1]
    else:
        return text


In [7]:
dataset_name = "msmarco_passage"
dataset = pt.get_dataset(dataset_name)

In [ ]:
index_path = "../dense_indexes/ffindex_" + dataset_name + "_gte-base-en-v1.5.h5"
ff_index = OnDiskIndex(
    Path(index_path), dim=768, query_encoder=q_encoder, mode=Mode.MAXP, max_id_length=47
)

In [10]:
ff_indexer = Indexer(ff_index, d_encoder, batch_size=8)
ff_indexer.index_dicts(docs_iter(dataset))

0it [00:00, ?it/s]

0it [00:00, ?it/s]


HTTPError: 404 Client Error: The specified resource does not exist. for url: https://msmarco.blob.core.windows.net/msmarcoranking/collection.tar.gz

In [12]:
dataset_name = "irds:msmarco-passage"
dataset = pt.get_dataset(dataset_name)

In [14]:
from pathlib import Path
from fast_forward import OnDiskIndex, Mode, Indexer

index_path = "../dense_indexes/ffindex_" + format_dataset_name(dataset_name) + "_gte-base-en-v1.5.h5"
ff_index = OnDiskIndex(
    Path(index_path), dim=768, query_encoder=q_encoder, mode=Mode.MAXP, max_id_length=47
)

In [ ]:
ff_indexer = Indexer(ff_index, d_encoder, batch_size=8)
ff_indexer.index_dicts(docs_iter(dataset))

0it [00:00, ?it/s]
                  cuments:   0%|          | 0/8841823 [00:00<?, ?it/s]
                                                                   [INFO] Please confirm you agree to the MSMARCO data usage agreement found at <http://www.msmarco.org/dataset.aspx>
0it [00:00, ?it/s]
                  cuments:   0%|          | 0/8841823 [00:00<?, ?it/s]
                                                                   [INFO] If you have a local copy of https://msmarco.z22.web.core.windows.net/msmarcoranking/collectionandqueries.tar.gz, you can symlink it here to avoid downloading it again: /home/anistor/.ir_datasets/downloads/31644046b18952c1386cd4564ba2ae69
0it [00:00, ?it/s]
                  cuments:   0%|          | 0/8841823 [00:00<?, ?it/s]
                                                                   [INFO] [starting] https://msmarco.z22.web.core.windows.net/msmarcoranking/collectionandqueries.tar.gz
0it [00:00, ?it/s]
msmarco-passage documents:   0%|          | 0/88